In [0]:
!pip install -q kaggle
import time
import zipfile
import os
import json
import cv2
import keras

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
!pip install --upgrade keras
!pip install mrcnn

In [0]:
keras.__version__

In [0]:
key = input("Please put your Kaggle credentials in JSON format in")
with open("kaggle.json", "w") as f:
    f.write(key) 

In [0]:
ROOT_DIR = os.path.abspath("/content/")
root_dir = "/content/"
MODEL_DIR = os.path.join(ROOT_DIR, "/content/")

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
start = time.time()
!kaggle competitions download airbus-ship-detection
print("downloading dataset took {} seconds".format(time.time()-start))
download_spent = time.time() - start

In [0]:
force = False #@param {type:"boolean"}
if download_spent > 50 or force == True:
  for file in [root_dir+"test_v2.zip",root_dir+"train_ship_segmentations_v2.csv.zip",root_dir+"train_v2.zip"]:
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()
      print(file)
else:
  print("Not unzipping because old dataset is still present, use force=True to stil unzip")

In [0]:
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [0]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [0]:
df = pd.read_csv(root_dir+"train_ship_segmentations_v2.csv").dropna()
df.index = np.arange(0, len(df))
display(df[:10])
print(df.ImageId[0])

In [0]:
from skimage.segmentation import mark_boundaries
from skimage.measure import label, regionprops

def masks_as_image(in_mask_list, all_masks=None):
    # Take the individual ship masks and create a single mask array for all ships
    if all_masks is None:
        all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)
  
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [0]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from tqdm import tqdm

In [0]:
only_ships_df = df[df.EncodedPixels.isnull()==False]

In [0]:
class ShipsDatasetOne(mrcnn.utils.Dataset):
  def load_dataset(self, dataset_frame, width, height):
    self.dataset_frame = dataset_frame
    self.add_class("ships", 1, "ship")
    count = 0
    for x in tqdm(dataset_frame.ImageId.unique()):
      ships = []
      for b in range(len(df.query('ImageId=="'+x+'"')['EncodedPixels'])):
        ships.append("ship")
      self.add_image("ships", image_id=count, path=x,
                    width=width, height=height,
                    ships=ships)
      count+=1
      
  
  def load_image(self,image_id):
    info = self.image_info[image_id]
    if info["id"] == image_id:
      ships = info['ships']
      filename = info['path']
      image = cv2.imread(root_dir+filename)
      return image
    else:
      print("FATAL ERROR! NOT MATHCING IDs")
  
  def load_mask(self,image_id):
    info = self.image_info[image_id]
    assert info["id"] == image_id
    ships = info['ships']
    filename = info['path']
    rle_0 = df.query('ImageId=="'+filename+'"')['EncodedPixels']
    mask = masks_as_image(rle_0) 
    class_ids = np.array([self.class_names.index(s) for s in ships])
    if len(class_ids) != 0:
      class_ids = np.array([1])
    return mask.astype(np.bool), class_ids.astype(np.int32)
  
  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['ships']

In [0]:
dataset_train = ShipsDatasetOne()
dataset_train.load_dataset(only_ships_df[:20000],768,768)

In [0]:
dataset_val = ShipsDatasetOne()
dataset_val.load_dataset(only_ships_df[-1000:],768,768)

In [0]:
dataset_val.prepare()
dataset_train.prepare()

###I  really dont know what the  problem was, but somehow np.unique([1]) != np.unique([1,1,1,1]), when using it for indexing of an ndarray
So i fixed it with:


```
if len(class_ids) != 0:
      class_ids = np.array([1])
 ```
Which just sets the class_ids to the shape of (1,) when the old shape is bigger than (0,)

So [1,1,1,1,1,1] -> [1]

This is not an issue because there is only class Id: 1 because we are only detecting ships, nothing else.


In [0]:
image_ids = np.random.choice(dataset_train.image_ids, 4)
print(image_ids)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [0]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "ships"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 ships

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 768

    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 2500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 10
    
config = ShapesConfig()

In [0]:
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [0]:
# Which weights to start with?
init_with = "coco"  # @param ["imagenet", "coco", "last"]

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    !wget -O mask_rcnn_coco.h5 https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers#to train by name pattern.
train = True # @param {type:'boolean'}
if train:
  model.train(dataset_train, dataset_val, 
              learning_rate=config.LEARNING_RATE, 
              epochs=1, 
              layers='heads')

In [0]:
if train:
  model.train(dataset_train, dataset_val, 
              learning_rate=config.LEARNING_RATE / 10,
              epochs=1, 
              layers="all")

In [0]:
model_path = os.path.join("mask_rcnn_ships.h5")
model.keras_model.save_weights(model_path)

In [0]:
download = True #@param {type:"boolean"}
if download:
  from google.colab import files
  files.download('mask_rcnn_ships.h5')

In [0]:
upload  = False #@param {type:"boolean"}
if upload:
  from google.colab import files
  uploaded = files.upload()

In [0]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(ROOT_DIR, "mask_rcnn_ships.h5")
#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


In [0]:
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [0]:
results = model.detect([original_image], verbose=1)

r = results[0]
print(r['masks'].shape)
print(r['class_ids'])
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [0]:
image_ids = np.random.choice(dataset_val.image_ids, 50)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [0]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [0]:
submit_df = pd.read_csv(root_dir+"sample_submission_v2.csv").dropna()
submit_df.index = np.arange(0, len(df))
display(submit_df[:10])

In [0]:
finished_d = {'ImageId': [], 'EncodedPixels': []}
finished_df = pd.DataFrame(data=finished_d)

In [0]:
for image in tqdm(submit_df.ImageId):
  img = cv2.imread(image)
  results = model.detect([img], verbose=0)
  r = results[0]
  c = False
  for x in range(len(r['class_ids'])):
    rle_0 = rle_encode(r['masks'][:,:,x])
    finished_df = finished_df.append({'ImageId': image, 'EncodedPixels': rle_0}, ignore_index=True)
    c = True
  if not c:
    finished_df = finished_df.append({'ImageId': image, 'EncodedPixels': ""}, ignore_index=True)
  

In [0]:
finished_df.to_csv('/content/submissions.csv')

In [0]:
!kaggle competitions submit -c airbus-ship-detection -f s/content/submissions.csv -m "Very first submission! \o/"

In [0]:
display(finished_df.head())

In [0]:
from google.colab import files
files.download('/content/submissions.csv')

In [0]:
predict_images = ["000155de5.jpg"]
for image in predict_images:
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
  img = cv2.imread(image)
  all_masks = []
  results = model.detect([img], verbose=0)
  r = results[0]
  for x in range(len(r['class_ids'])):
    all_masks.append(r['masks'][:,:,x])
  complete_mask = np.stack(all_masks, axis=2)
  ax1.imshow(img)
  ax1.set_title('Image')
  ax2.set_title('Mask')
  ax2.imshow(complete_mask[...,0], cmap='gray')
  plt.show()
  rle_0 = rle_encode(complete_mask)